Project 1 : Multiclass Multilabel prediction For stack overflow Questions
Download data from here : https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

Goal : Given text for Questions , predict tags associated with them

Suggested Guidelines :

1. Pick top 10 most occurring tags and filter data for them and build your model for them

2. You can expand your model later on

3. Since there is no performance threshold and associated data given to you , you'll need to separate data in train/validation yourself

4. Before directly jumping in with LSTM ( or any other model of your choice ) , explore this text data and see if any basic data cleaning is required .

5. Usage of Answers.csv is optional

6. This is a multi class - multilabel prediction problem , keep that in mind

7. Your end solution needs to be uploaded on GitHub repo of your team




Answer_data= Id	OwnerUserId	CreationDate	ParentId	Score	Body
tags= Id	Tag
questions= Id	OwnerUserId	CreationDate	ClosedDate	Score	Title	Body


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
questions_df= pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Questions.csv', encoding='latin-1' )

In [ ]:
# Display Data
questions_df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
questions_data= questions_df.drop(columns=['OwnerUserId','CreationDate',	'ClosedDate',	'Score'])
#df1 = df.drop(columns=['Name','Ticket'])
questions_data.head()

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
#Reads the tags file
Tags_df = pd.read_csv('/content/drive/MyDrive/Datasets/DL project Multiclass Multilabel prediction For stack overflow QuestionsPage Not completed: Project 1 : Multiclass Multilabel prediction For stack overflow Questions./Tags.csv', encoding='latin-1')
Tags_df.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [ ]:
# Identify top 10 tags by count
import collections
tagCount = collections.Counter(list(Tags_df['Tag'])).most_common(10)
print(tagCount)

[('javascript', 124155), ('java', 115212), ('c#', 101186), ('php', 98808), ('android', 90659), ('jquery', 78542), ('python', 64601), ('html', 58976), ('c++', 47591), ('ios', 47009)]


In [ ]:
# Manipulation the tags dataframe so that all the tags for an ID as a list in a row (grouped by Question ID)
def add_tags (questions_data_Id):
  return tagCount[tagCount['Id'] == question_data_id['Id']].Tag.values

# top10 = tagCount.apply(add_tags, axis=1)
top10 = Tags_df.groupby('Id')['Tag'].apply(list)

In [ ]:
# Merge the Questions and Tags data frame by ID
total=pd.merge(questions_data, top10, on='Id')
total.head()
#Our Dataset would now have only Id, Title, Body & Tags

,Id,Title,Body,Tag
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3, air]"
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net, sitemap]"
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors, color-s..."
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [ ]:
# Ensure that the Tag column only contains string values. Converting non-string values to strings using the astype method:

total['Tag'] = total['Tag'].astype(str)


# The Tag column contains lists of tags, but they are represented as strings, not actual lists.
# To convert the Tag column to a list of tags, you can use the str.split method:

total['Tag'] = total['Tag'].str.strip('[]').str.split(', ')

Step 2: Data handling

In [ ]:
# check missing value
print('True missing value :', total['Id'].isnull().sum().sum())

True missing value : 0


Text  Preprocessing usking lib- NTLK, spaCy

In [ ]:
import re  #For text pattern matching and manipulation
import nltk
nltk.download('stopwords')
nltk.download('punkt') # for tkenizer
from nltk.corpus import stopwords # A collection of common words like "the", "and", etc.
from nltk.tokenize import word_tokenize # Splits text into individual words
from nltk.stem import PorterStemmer # Reduces words to their base form

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def clean_text(text):
  # Remove HTML tags
  text = re.sub(r'<.*?>', '', text)
  # remove special characters and digits
  text = re.sub(r'\W+', ' ', text)
  # convert to lower()
  text= text.lower()
  # Tokenize and remove stopwords
  words = word_tokenize(text)
  words = [stemmer.stem(word) for word in words if word not in stop_words]
  return ' '.join(words)

# Apply cleaning to question text
total['text'] = total['Title'] + ' ' + total['Body']  # Combine Title and Body
total['text'] = total['text'].apply(clean_text)


RecursionError: maximum recursion depth exceeded in comparison

In [ ]:
# Tags preprocesing Use MultiLabelBinarizer from sklearn on the Class labels(Tags)
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(total['Tag'])
print(multilabel_binarizer.classes_)

In [ ]:
# One-hot encoding the tags
tags_encoded = multilabel_binarizer.transform(total['Tag'])

# Convert encoded tags to a DataFrame
tags_df = pd.DataFrame(tags_encoded, columns=multilabel_binarizer.classes_)


Multiclass Multilabel Setup

In [ ]:
from sklearn.model_selection import train_test_split
X = total[['Title', 'Body']]
y = tags_encoded

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)



Model Building

In [ ]:
#Text Vectorization- Convert the text into numerical vectors using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Use TF-IDF vectorizer to convert text to numerical data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)


NameError: name 'X_train' is not defined

Model Selection
We can use different models for multiclass multilabel prediction:

Logistic Regression for each label.
LSTM (if you're using deep learning models).

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
model = MultiOutputClassifier(RandomForestClassifier())
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_val_tfidf)


 Evaluation- Use metrics like precision, recall, F1-score, and accuracy to evaluate the performance

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred, target_names=mlb.classes_))

              precision    recall  f1-score   support

          c#       0.00      0.00      0.00         0
        java       0.00      0.00      0.00         0
         php       0.00      0.00      0.00         0
  javascript       0.00      0.00      0.00         0
      jquery       0.00      0.00      0.00         0
     android       0.00      0.00      0.00         0
         c++       0.00      0.00      0.00         0
      iphone       0.00      0.00      0.00         0
      python       0.00      0.00      0.00         0
     asp.net       0.00      0.00      0.00         0

   micro avg       0.00      0.00      0.00         0
   macro avg       0.00      0.00      0.00         0
weighted avg       0.00      0.00      0.00         0
 samples avg       0.00      0.00      0.00         0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471:

Summary of Steps:
* Data Preprocessing: Handle missing values, clean the text, and filter for top 10 tags.
* Feature Engineering: Combine the question Title and Body, and extract date and numeric features.
* Multilabel Encoding: Convert tags into a binary format for multiclass multilabel classification.
* Model Building: Use text vectorization (e.g., TF-IDF) and build a machine learning
 * model (Logistic Regression or LSTM).
Evaluation: Assess your model using appropriate metrics.

# Choosing LMST for Deep learning Model

* Data Preprocessing for LSTM-
 LSTM models require numeric sequences as input. First, you’ll need to tokenize and pad the text data.

(a) Tokenize and Pad the Text
Use Keras to tokenize the text and create sequences of tokens. Then, pad these sequences to ensure all inputs have the same length.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set parameters
MAX_NUM_WORDS = 5000  # Maximum number of words in the vocabulary
MAX_SEQUENCE_LENGTH = 100  # Maximum sequence length

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(questions_df['text'])

# Convert text to sequences
X_sequences = tokenizer.texts_to_sequences(questions_df['text'])

# Pad sequences to ensure uniform length
X_padded = pad_sequences(X_sequences, maxlen=MAX_SEQUENCE_LENGTH)


Step 5: Multilabel Binarization- Transform the tags into a multilabel binary format.
 One-hot encode the tags to handle the multilabel classification.

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Use MultiLabelBinarizer to convert tags to a binary matrix
mlb = MultiLabelBinarizer(classes= top_tags)
y_encoded= mlb.fit_transform(questions_df['text'])

# Convert binary tags to a DataFrame for easy access
#tags_df = pd.DataFrame(y_encoded, columns=mlb.classes_)

# Add binary tags to the original dataframe
#questions_df = pd.concat([questions_df, tags_df], axis=1)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ª', '²', '³', 'µ', '¹', 'º', '¼', '½', '¾', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ'] will be ignored
  warnings.warn(


Step 6: Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)



 Building the LSTM Model
 LSTMs are commonly used for text sequences, and they are effective for sequential data like text. LSTM-based neural network using Keras.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Set model parameters
EMBEDDING_DIM = 100  # Dimension of the embedding space
LSTM_UNITS = 128     # Number of LSTM units
DROPOUT_RATE = 0.5   # Dropout rate for regularization
OUTPUT_UNITS = y_encoded.shape[1]  # Number of output units (same as number of tags)

# Build the model
model = Sequential()

# Add an embedding layer
model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))

# Add LSTM layer
model.add(LSTM(LSTM_UNITS, return_sequences=False))

# Add Dropout layer for regularization
model.add(Dropout(DROPOUT_RATE))

# Add Dense layer with sigmoid activation for multilabel classification
model.add(Dense(OUTPUT_UNITS, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=10,  # You can adjust the number of epochs
                    batch_size=32,  # You can adjust the batch size
                    validation_data=(X_val, y_val))


Epoch 1/10
15773/15773 ━━━━━━━━━━━━━━━━━━━━ 3295s 209ms/step - accuracy: 0.0310 - loss: 0.0045 - val_accuracy: 0.0000e+00 - val_loss: 1.4885e-09
Epoch 2/10
15773/15773 ━━━━━━━━━━━━━━━━━━━━ 3240s 205ms/step - accuracy: 0.0574 - loss: 6.6661e-09 - val_accuracy: 0.0000e+00 - val_loss: 4.3559e-11
Epoch 3/10
13001/15773 ━━━━━━━━━━━━━━━━━━━━ 8:46 190ms/step - accuracy: 0.0936 - loss: 6.9936e-10

In [ ]:
# Evaluate the model

from sklearn.metrics import classification_report

# Make predictions on validation set
y_pred = model.predict(X_val)

# Convert predictions to binary values (1 if prediction > 0.5, else 0)
y_pred_binary = (y_pred > 0.5).astype(int)

# Evaluate the model using precision, recall, and F1-score
print(classification_report(y_val, y_pred_binary, target_names=mlb.classes_))


* Hyperparameter Tuning: Experiment with the number of LSTM units, dropout rates, learning rates, and embedding dimensions to improve the model.
* Early Stopping: Implement early stopping to avoid overfitting.

In [ ]:


from tensorflow.keras.callbacks import EarlyStopping

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])


Summary:
* Preprocess Text: Clean, tokenize, and pad the text.
* Multilabel Encoding: Convert tags into a binary matrix.
* Build LSTM Model: Create a sequential model with an embedding layer, LSTM units, and a final dense layer with sigmoid activation.
* Train the Model: Train using binary_crossentropy loss and validate on the validation set.
* Evaluate the Model: Measure performance using appropriate classification metrics.